In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import torch
from torch.nn import Embedding, Linear, LSTM, Module
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from tqdm import tqdm

import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [2]:
!pip install google.colab

^C


In [4]:
full_dictionary_location = "words_250000_train.txt"
def build_dictionary(dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

full_dictionary = build_dictionary(full_dictionary_location)
full_dictionary

['aaa',
 'aaaaaa',
 'aaas',
 'aachen',
 'aaee',
 'aag',
 'aahed',
 'aahs',
 'aal',
 'aalesund',
 'aaliis',
 'aalst',
 'aam',
 'aandahl',
 'aao',
 'aapss',
 'aar',
 'aarau',
 'aardvark',
 'aardwolf',
 'aaren',
 'aargh',
 'aarika',
 'aaronic',
 'aaronite',
 'aaronsburg',
 'aarp',
 'aarrghh',
 'aas',
 'aasvogels',
 'aaup',
 'aavso',
 'aba',
 'ababdeh',
 'abac',
 'abacas',
 'abacaxi',
 'abaci',
 'abacination',
 'abaciscus',
 'aback',
 'abaco',
 'abacterial',
 'abactinally',
 'abactor',
 'abaculus',
 'abacuses',
 'abada',
 'abaddon',
 'abadengo',
 'abadite',
 'abaft',
 'abagail',
 'abailard',
 'abaised',
 'abaisse',
 'abaka',
 'abakas',
 'abalation',
 'abalienated',
 'abalienation',
 'abalones',
 'abamp',
 'abamperes',
 'abana',
 'abandon',
 'abandoned',
 'abandonee',
 'abandoners',
 'abandonment',
 'abandons',
 'abanet',
 'abanic',
 'abantes',
 'abaptiston',
 'abarambo',
 'abaris',
 'abarticular',
 'abas',
 'abased',
 'abasedness',
 'abasements',
 'abasers',
 'abasgi',
 'abashed',
 'abashe

In [5]:
full_dictionary = [ ('<'+ w +'>') for w in full_dictionary]

Now we have the text, encode it as integers.

In [6]:
concat_full_dictionary = full_dictionary
random.shuffle(concat_full_dictionary)
concat_full_dictionary = ''.join(concat_full_dictionary)
concat_full_dictionary

'<noshed><symbiote><myrcia><domremylapucelle><isonzo><broadset><batters><strepsitene><attentive><fliting><royston><undiagnosable><virulentness><prohibitionist><phytoptidae><unroyalized><agronomically><unamputative><infelicity><sooting><clu><unthrifty><elinvar><agrements><illguide><cypriots><daggerlike><swashing><supergravitation><whing><coolabah><exoticism><interreceived><jamborees><decretory><tummer><escobadura><pollinations><hysteromyomectomy><ehlite><thirstmaddened><fugue><sickouts><booz><usurp><wellwilled><matronize><cylindricoblong><passional><objectively><eponymism><boreal><archicerebra><outwring><wherehence><unremittedly><autocatheterism><disgrace><indiscriminateness><mitchel><ferritungstite><schizogregarinida><twiceexcelled><stadle><selfangry><corantijn><crudded><vinaigretted><pupilless><diplomatical><enterocentesis><cavorter><marts><trigyn><stridulant><gamboges><refracturable><buckner><fosh><ringleted><nucleon><morphinomania><unstealthy><calp><drivehead><catwise><essed><sacrif

In [7]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
text = concat_full_dictionary
chars = tuple(set(text))
print(chars)
int2char = dict(enumerate(chars))
print(int2char)
char2int = {ch: ii for ii, ch in int2char.items()}
print(char2int)
encoded = np.array([char2int[ch] for ch in text])
encoded

('g', 'v', 'q', 'x', 'y', 't', 'c', 'r', '<', 'k', 'p', 'z', 'u', 'n', 'a', 'j', 'o', 'w', 's', 'e', 'd', 'f', '>', 'm', 'l', 'i', 'b', 'h')
{0: 'g', 1: 'v', 2: 'q', 3: 'x', 4: 'y', 5: 't', 6: 'c', 7: 'r', 8: '<', 9: 'k', 10: 'p', 11: 'z', 12: 'u', 13: 'n', 14: 'a', 15: 'j', 16: 'o', 17: 'w', 18: 's', 19: 'e', 20: 'd', 21: 'f', 22: '>', 23: 'm', 24: 'l', 25: 'i', 26: 'b', 27: 'h'}
{'g': 0, 'v': 1, 'q': 2, 'x': 3, 'y': 4, 't': 5, 'c': 6, 'r': 7, '<': 8, 'k': 9, 'p': 10, 'z': 11, 'u': 12, 'n': 13, 'a': 14, 'j': 15, 'o': 16, 'w': 17, 's': 18, 'e': 19, 'd': 20, 'f': 21, '>': 22, 'm': 23, 'l': 24, 'i': 25, 'b': 26, 'h': 27}


array([ 8, 13, 16, ...,  5, 19, 22])

In [8]:
text[:100]

'<noshed><symbiote><myrcia><domremylapucelle><isonzo><broadset><batters><strepsitene><attentive><flit'

In [9]:
encoded[:100]

array([ 8, 13, 16, 18, 27, 19, 20, 22,  8, 18,  4, 23, 26, 25, 16,  5, 19,
       22,  8, 23,  4,  7,  6, 25, 14, 22,  8, 20, 16, 23,  7, 19, 23,  4,
       24, 14, 10, 12,  6, 19, 24, 24, 19, 22,  8, 25, 18, 16, 13, 11, 16,
       22,  8, 26,  7, 16, 14, 20, 18, 19,  5, 22,  8, 26, 14,  5,  5, 19,
        7, 18, 22,  8, 18,  5,  7, 19, 10, 18, 25,  5, 19, 13, 19, 22,  8,
       14,  5,  5, 19, 13,  5, 25,  1, 19, 22,  8, 21, 24, 25,  5])

In [10]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [11]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        
        # The features
        x = arr[:, n:n+n_steps]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [12]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [13]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 8 13 16 18 27 19 20 22  8 18]
 [10 16 18  5  6 16 13 20 25  5]
 [22  8 14 12  5 16 25 13 27 25]
 [12 23 22  8 14 12  5 16  5  4]
 [ 7 19 22  8 18 25 23 16 13 18]
 [19 22  8 23 14  0 25 18  5 19]
 [22  8 10 19  7  0 19 24 25 18]
 [22  8 18 16 23 14  5 16  6  4]
 [10 25  5 25 18 22  8  7 16 23]
 [16  5 22  8 25 13  5  7 14 10]]

y
 [[13 16 18 27 19 20 22  8 18  4]
 [16 18  5  6 16 13 20 25  5 25]
 [ 8 14 12  5 16 25 13 27 25 26]
 [23 22  8 14 12  5 16  5  4 10]
 [19 22  8 18 25 23 16 13 18 19]
 [22  8 23 14  0 25 18  5 19  7]
 [ 8 10 19  7  0 19 24 25 18 16]
 [ 8 18 16 23 14  5 16  6  4 18]
 [25  5 25 18 22  8  7 16 23 23]
 [ 5 22  8 25 13  5  7 14 10 19]]


In [13]:
np.random.choice?

In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## Define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # Initialize the weights
        self.init_weights()
      
    
    def forward(self, x, hc):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        
        ## Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        ## Ppass x through the dropout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        #x = x.view(x.size()[0]*x.size()[1], self.n_hidden)
        torch.reshape(x,(x.size()[0]*x.size()[1], self.n_hidden))

        ## Put x through the fully-connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, size = None , replace = True , p=p/p.sum())
            
        return self.int2char[char], h
    def predict_prob_dist(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = []
        prob = []
        for i in range(len(self.chars)):
          prob[i] = p[i]/p.sum()
          z = [top_ch[i] , prob[i]] 
          char[i] = z
        for pair in char:
          pair = [self.int2char[pair[0]] , pair[1]]
            
        return char, h
    

    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())
        

In [15]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    for e in range(epochs):
        
        h = net.init_hidden(n_seqs)
        
        for x, y in get_batches(data, n_seqs, n_steps):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            
            t = targets
            t.type(torch.LongTensor)
            #targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor)
            torch.reshape(t ,(n_seqs*n_steps,))
            loss = criterion(output, t)

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)

                    t = targets
                    t.type(torch.LongTensor)
                    #targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor)
                    torch.reshape(t ,(n_seqs*n_steps,))
                    val_loss = criterion(output, t)
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [20]:
if 'net' in locals():
    del net

In [21]:
# Initialize and print the network
net = CharRNN(chars, n_hidden=256, n_layers=2)

print(net)

CharRNN(
  (lstm): LSTM(28, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=28, bias=True)
)


In [18]:
n_seqs, n_steps = 15, 28

train(net, encoded, epochs=25, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=10)

Streaming output truncated to the last 5000 lines.
Epoch: 16/25... Step: 88180... Loss: 3.1477... Val Loss: 3.1446
Epoch: 16/25... Step: 88190... Loss: 3.1581... Val Loss: 3.1416
Epoch: 16/25... Step: 88200... Loss: 3.1416... Val Loss: 3.1407
Epoch: 16/25... Step: 88210... Loss: 3.1759... Val Loss: 3.1438
Epoch: 16/25... Step: 88220... Loss: 3.1097... Val Loss: 3.1442
Epoch: 16/25... Step: 88230... Loss: 3.1271... Val Loss: 3.1425
Epoch: 16/25... Step: 88240... Loss: 3.1347... Val Loss: 3.1417
Epoch: 16/25... Step: 88250... Loss: 3.1613... Val Loss: 3.1411
Epoch: 16/25... Step: 88260... Loss: 3.0764... Val Loss: 3.1413
Epoch: 16/25... Step: 88270... Loss: 3.1520... Val Loss: 3.1417
Epoch: 16/25... Step: 88280... Loss: 3.1338... Val Loss: 3.1417
Epoch: 16/25... Step: 88290... Loss: 3.1616... Val Loss: 3.1417
Epoch: 16/25... Step: 88300... Loss: 3.1329... Val Loss: 3.1423
Epoch: 16/25... Step: 88310... Loss: 3.1276... Val Loss: 3.1414
Epoch: 16/25... Step: 88320... Loss: 3.1263... Val Lo

In [22]:
# change the name, for saving multiple files
model_name = 'rnn_2_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [30]:
def sample(net, size, prime='a', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    
    h = net.init_hidden(1)
    
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [39]:
np.random.choice?

In [31]:
print(sample(net, 0, prime='a', top_k=None, cuda=False))

af


## Loading a checkpoint

In [27]:
torch.load?

In [28]:
# Here we have loaded in a model that trained over 1 epoch `rnn_1_epoch.net`
with open('rnn_1_epoch.net', 'rb') as f:
    checkpoint = torch.load(f,map_location=torch.device('cpu'))
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [32]:
# Change cuda to True if you are using GPU!
print(sample(loaded, 0, cuda=False, top_k=None, prime="m"))

me
